<h1 align="center"><font size="5">Segmenting and Clustering Neighborhoods in Toronto - Week 3 Assignment</font></h1>

#### Importing the necessary packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


### Reading the Wikipage into a list

##### The table that is needed is encapsulated in tbody tags. and table rows are encapsulated in  tr tags. For each row, when we just take  the text, we will see that each value is in a newline of it's own. I will use \n to split the row into individual cells, but before doing that I am adding ", " before the newline character. This will come in handy when we need to group neighborhoods by postal code and Boroughs

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('tbody')

RowList = []

for rows in table.find_all('tr'):
    row = rows.text.replace('\n', ', \n')
    RowList.append(row.split('\n'))


#### Now, I am reading list I collected above into a data frame and assigning column names. Delete the empty rows and the one row that has column names. I am also dropping all rows where Borough is 'Not assigned'

In [3]:
df = pd.DataFrame(RowList, columns=['empty', 'PostalCode','Borough','Neighbourhood','empty2'])
df.drop(columns=['empty','empty2'], inplace=True)
df.drop(0, inplace=True)
df.drop(df[df.Borough=='Not assigned, '].index, axis=0, inplace=True)
df.shape

(212, 3)

#### If Neighborhood is not assigned, then update Neighbourhood value to Borough value. There is only one row of that after deleting all rows where Borough is not assigned.

In [4]:
df['Neighbourhood'] == 'Not assigned, '

3      False
4      False
5      False
6      False
7      False
8      False
9       True
11     False
12     False
13     False
15     False
16     False
17     False
18     False
19     False
20     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
32     False
33     False
34     False
35     False
36     False
39     False
       ...  
237    False
238    False
241    False
242    False
245    False
246    False
247    False
248    False
251    False
252    False
253    False
254    False
257    False
258    False
259    False
264    False
267    False
268    False
269    False
270    False
271    False
272    False
273    False
274    False
275    False
284    False
285    False
286    False
287    False
288    False
Name: Neighbourhood, Length: 212, dtype: bool

In [5]:
df.loc[df['Neighbourhood'] == 'Not assigned, ', 'Neighbourhood'] = df['Borough']

#### Search for 'Not Assigned' Neighborhood and confirm it returns no results

In [6]:
df['Neighbourhood'].where(df['Neighbourhood'] == 'Not assigned, ')

3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
11     NaN
12     NaN
13     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
32     NaN
33     NaN
34     NaN
35     NaN
36     NaN
39     NaN
      ... 
237    NaN
238    NaN
241    NaN
242    NaN
245    NaN
246    NaN
247    NaN
248    NaN
251    NaN
252    NaN
253    NaN
254    NaN
257    NaN
258    NaN
259    NaN
264    NaN
267    NaN
268    NaN
269    NaN
270    NaN
271    NaN
272    NaN
273    NaN
274    NaN
275    NaN
284    NaN
285    NaN
286    NaN
287    NaN
288    NaN
Name: Neighbourhood, Length: 212, dtype: object

#### Add the Neighborhoods if Postcode and Borough are the same. The comma that we added above comes in handy now to separate different neighbourhoods in the same row. However, we need to strip the last character in each column as that is a comma.

In [7]:
newdf = df.groupby(['PostalCode','Borough']).sum()
newdf=newdf.reset_index()
newdf.PostalCode = newdf.PostalCode.apply(lambda x:x[:-2])
newdf.Borough = newdf.Borough.apply(lambda x: x[:-2])
newdf.Neighbourhood = newdf.Neighbourhood.apply(lambda x: x[:-2])

newdf

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### The newcount below reflects the fact that mutliple neighborhoods are combined in one row.

In [8]:
newdf.shape

(103, 3)

In [9]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 7.2MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

In [11]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postalcode = 'M1A'
# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('Toronto, Ontario'.format(postalcode))
    print (g)
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print (lat_lng_coords)

<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
<[OVER_QUERY_LIMIT] Google - Geoco

KeyboardInterrupt: 

In [10]:
lldf = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
finaldf = pd.merge(newdf, lldf, how='inner',left_on = 'PostalCode', right_on = 'Postal Code')

In [12]:
finaldf.drop(['Postal Code'],axis=1, inplace=True)

In [13]:
finaldf

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [14]:
!pip install folium

    100% |████████████████████████████████| 92kB 6.3MB/s eta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

In [15]:
import folium
from sklearn.cluster import KMeans
import matplotlib as plt

In [16]:
map_toronto = folium.Map(location =[43.6532,-79.3832],zoom_start=15)
map_toronto

In [17]:
for PostalCode,Borough,Neighbourhood,Latitude,Longitude in zip(finaldf['PostalCode'],finaldf['Borough'],finaldf['Neighbourhood'],finaldf['Latitude'],finaldf['Longitude']):
    label = PostalCode
    folium.CircleMarker(
        [Latitude,Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
map_toronto

In [3]:
# The code was removed by Watson Studio for sharing.

In [24]:
# let's explore Downtown Toronto
latitude = 43.669542
longitude = -79.422564
limit = 100
radius=500

In [25]:

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
latitude,
longitude,
radius,
limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=V3W3HMNOVROVANSHDGN3ZJM4RYRDYCQPRL51JHFJ3YBOOVMO&client_secret=KXOGU4CNMAUXKDGPWYPGQAIIPU0E3XHVRXWI1UPMYH25UE4P&v=20190115&ll=43.669542,-79.422564&radius=500&limit=100'

In [26]:
import requests
results = requests.get(url).json()

In [28]:
import json
import pandas as pd

list =[]
new_results= results['response']['groups']
#print(new_results)
for level1 in new_results:
    for level2 in level1['items']:
        for level3 in level2['venue']['categories']:
            category = level3['shortName']
        name = level2['venue']['name']
        longitude=level2['venue']['location']['lng']
        latitude = level2['venue']['location']['lat']
        city = level2['venue']['location']['city']
        print(postalCode)
        list.append([category,name,longitude,latitude,city])
        print(list)
df = pd.DataFrame(list,columns=['Category','Name','Longitude','Latitude','city'])
df

M6G 3B6
[['Grocery Store', 'Fiesta Farms', -79.42048512748114, 43.66847077052224, 'Toronto', 'M6G 3B6']]


KeyError: 'postalCode'

In [44]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venuesnearby = results['response']['groups'][0]['items'][0]['venue']
venuesnearby =json_normalize(venuesnearby)
venuesnearby

,categories,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,photos.count,photos.groups
0,"[{'name': 'Nightclub', 'pluralName': 'Nightclu...",4de939f5814d89ee7e821c5d,50 Cumberland Street,CA,Toronto,Canada,Between Yonge and Bay,36,"[50 Cumberland Street (Between Yonge and Bay),...","[{'lng': -79.42283025079402, 'label': 'display...",43.669272,-79.42283,M4W 1J5,ON,Empire Restaurant and Lounge,0,[]


In [64]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venuesnearby = results['response']['groups'][0]['items']
venuesnearby=json_normalize(venuesnearby)
newdf = venuesnearby.filter(['venue.categories','venue.location.address','venue.location.lat','venue.location.lng','venue.name'],axis=1)
newdf

,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.name
0,"[{'primary': True, 'name': 'Grocery Store', 'i...",200 Christie St,43.668471,-79.420485,Fiesta Farms
1,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",1028 Shaw St,43.669107,-79.426105,Contra Cafe
2,"[{'primary': True, 'name': 'Coffee Shop', 'id'...",657 Dupont St,43.671585,-79.421366,Starbucks
3,"[{'primary': True, 'name': 'Italian Restaurant...",787 Dupont Street,43.670679,-79.426148,Vinny’s Panini
4,"[{'primary': True, 'name': 'Diner', 'id': '4bf...",1071 Shaw St.,43.670550,-79.426541,Universal Grill
5,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",146 Christie Street,43.667360,-79.419938,Scout and Cash Caffe
6,"[{'primary': True, 'name': 'Grocery Store', 'i...",650 Dupont St,43.671807,-79.421102,Loblaws
7,"[{'primary': True, 'name': 'Restaurant', 'id':...",971 Ossington Ave,43.667858,-79.428054,Actinolite
8,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",672 Dupont Street,43.672003,-79.422209,Faema Caffe
9,"[{'primary': True, 'name': 'Baby Store', 'id':...",631 Dupont St,43.671824,-79.420499,Marlenes Just Babies


ValueError: labels ['venue.location.formattedAddress' 'venue.location.labeledLatLngs'
 'venue.location.state' 'venue.photos.count' 'venue.photos.groups'
 'venue.venuePage.id'] not contained in axis

In [60]:
venuesnearby.drop()

,reasons.count,venue.categories,venue.location.address,venue.location.crossStreet,venue.location.distance,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.name
0,0,"[{'primary': True, 'name': 'Grocery Store', 'i...",200 Christie St,at Essex St,205,43.668471,-79.420485,M6G 3B6,Fiesta Farms
1,0,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",1028 Shaw St,Yarmouth,289,43.669107,-79.426105,NaN,Contra Cafe
2,0,"[{'primary': True, 'name': 'Coffee Shop', 'id'...",657 Dupont St,at Christie St.,247,43.671585,-79.421366,M6G 1Z4,Starbucks
3,0,"[{'primary': True, 'name': 'Italian Restaurant...",787 Dupont Street,NaN,315,43.670679,-79.426148,M6G 1Z5,Vinny’s Panini
4,0,"[{'primary': True, 'name': 'Diner', 'id': '4bf...",1071 Shaw St.,Dupont,339,43.670550,-79.426541,NaN,Universal Grill
5,0,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",146 Christie Street,Bloor,322,43.667360,-79.419938,M6G 3B3,Scout and Cash Caffe
6,0,"[{'primary': True, 'name': 'Grocery Store', 'i...",650 Dupont St,at Christie St.,278,43.671807,-79.421102,M6G 4B1,Loblaws
7,0,"[{'primary': True, 'name': 'Restaurant', 'id':...",971 Ossington Ave,NaN,480,43.667858,-79.428054,M6G 3V5,Actinolite
8,0,"[{'primary': True, 'name': 'Café', 'id': '4bf5...",672 Dupont Street,Christie,275,43.672003,-79.422209,M6G 1Z6,Faema Caffe
9,0,"[{'primary': True, 'name': 'Baby Store', 'id':...",631 Dupont St,NaN,303,43.671824,-79.420499,M6G 1Z4,Marlenes Just Babies
